We are training our word2vec models here based on the Blodgett dataset. 
We are also using a pre-trained word2vec model on the Google News 300 dataset. 

Blodgett_dict.json and google_news_dict.json are the dictionaries that are created by doing this, but we have uploaded a copies of the same in our supplementary materials.

LIWC_swear_dictionary.txt is the LIWC dictionary that was used by the authors in the original paper and we also used the same. This documentation for LIWC2007 can be found at: http://www.gruberpeplab.com/teaching/psych231_fall2013/documents/231_Pennebaker2007.pdf
However, we have also uploaded a copy of this dictionary in our supplementary materials. 

In [ ]:
import gensim
import gensim.downloader
import json
import pandas as pd

In [ ]:
# Uncomment the first line if you don't have already downloaded the gensim-data which contains the pretrained word2vec model

#model_gn = gensim.downloader.load('word2vec-google-news-300')
model_gn = gensim.models.KeyedVectors.load_word2vec_format("/home/cc/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz", binary=True) 
replace_dict_1 = {}


with open("/home/cc/LIWC_swear_dictionary.txt") as f:
  lines = f.readlines()
  word_set = set([l.split()[0] for l in lines])

for key in word_set:
  try:
    val = model_gn.most_similar(key,topn=20)[0][0]
    i = 1
    while val in key or key in val or val in word_set:
      val = model_gn.most_similar(key)[i][0]
      i += 1
    replace_dict_1[key]=val
      
  except (KeyError, IndexError):
    pass

In [ ]:
with open("google_news_dict.json","w") as f:
  json.dump(replace_dict_1,f)

#Blodget_dictionary

In [ ]:
# Loading the dataset
df = pd.read_csv("Blodgett_500k.csv")

#preprocessing and cleaning the text data that has to be trained

review_text = df.tweet_text.apply(gensim.utils.simple_preprocess)

#loading the model

model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

#Building vocab

model.build_vocab(review_text, progress_per=1000)

#training the word2vec model

model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)


(16880938, 21629045)

In [ ]:
replace_dict_2 = {}

with open("LIWC_swear_dictionary.txt") as f:
  lines = f.readlines()
  word_set = set([l.split()[0] for l in lines])

for key in word_set:
  try:
    val = model.wv.most_similar(key)[0][0]
    i = 1
    while val in key or key in val or val in word_set:
      val = model.wv.most_similar(key)[i][0]
      i += 1
    replace_dict_2[key]=val
      
  except (KeyError, IndexError):
    #replace_dict_2[key] = "None"
    pass

In [ ]:
with open("Blodgett_dict.json","w") as f:
  json.dump(replace_dict_2,f)